In [8]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import time
import warnings
import torch.nn as nn
import pickle
from dataset_loader import *
from utils import *
import pandas as pd
from datasets import Dataset, DatasetDict
from retrain_utils import *

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
dataset_name = 'corona'
ds = get_ds(dataset_name)

In [11]:
model_name =  'huawei-noah/TinyBERT_General_4L_312D'
folder_name = 'tinybert'

In [ ]:
retrain_utils = RetrainUtils(model_name, dataset_name)
added_train = retrain_utils.replace_sentences(ds['train'].to_pandas(), RetrainAction.ADD)
removed_train = retrain_utils.replace_sentences(ds['train'].to_pandas(), RetrainAction.REMOVE)
replaced_train = retrain_utils.replace_sentences(ds['train'].to_pandas(), RetrainAction.REPLACE)

In [17]:
ds['train'] = replaced_train

In [18]:
# notice is is train to test
ds['test'] = get_ds('counter')['train']

In [12]:
set_seed()
model = load_model(model_name)
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}/check', num_train_epochs=10)
model.save_pretrained(f'{folder_name}/{dataset_name}/updated_model')

loading configuration file config.json from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312D/snapshots/34707a33cd59a94ecde241ac209bf35103691b43/config.json
Model config BertConfig {
  "_name_or_path": "huawei-noah/TinyBERT_General_4L_312D",
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312

  0%|          | 0/8000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

PyTorch: setting up devices
***** Running training *****
  Num examples = 8000
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2500


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.434170,0.816000
2,0.377200,0.285740,0.885000
3,0.377200,0.319784,0.886000
4,0.186500,0.333895,0.889000
5,0.186500,0.442000,0.883500
6,0.111600,0.407626,0.898500
7,0.111600,0.438179,0.897000
8,0.069300,0.504073,0.897500
9,0.069300,0.548495,0.894500
10,0.047800,0.553073,0.891500


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to tinybert/corona/check/checkpoint-250
Configuration saved in tinybert/corona/check/checkpoint-250/config.json
Model weights saved in tinybert/corona/check/checkpoint-250/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to tinybert/corona/check/checkpoint-500
Configuration saved in tinybert/corona/check/checkpoint-500/config.json
Model weights saved in tinybert/corona/check/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to tinybert/corona/check/checkpoint-750
Configuration saved in tinybert/corona/check/checkpoint-750/config.json
Model weights saved in tinybert/corona/check/checkpoint-750/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to tinybert/corona/check/checkpoint-1000
Configura

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312D/snapshots/34707a33cd59a94ecde241ac209bf35103691b43/config.json
Model config BertConfig {
  "_name_or_path": "huawei-noah/TinyBERT_General_4L_312D",
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /ho

In [7]:
def predict_sentences(sentences):
    encoded = [[101] +[tokenizer._convert_token_to_id_with_added_voc(token) for token in tokens] + [102]         
               for tokens in sentences]
    #encoded = tokenizer.encode(sentences, add_special_tokens=True, return_tensors="pt").to(device)
    to_pred = torch.tensor(encoded, device=device)
    outputs = traced_model(to_pred)[0]
    print(outputs)
    return torch.argmax(outputs, dim=1).cpu().numpy()

### save best model's folder as 'model'

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(f'{folder_name}/{dataset_name}/updated_model', num_labels=2, torchscript=True).to(device)
model = model.eval()

loading configuration file tinybert/sentiment/updated_model/config.json
Model config BertConfig {
  "_name_or_path": "tinybert/sentiment/updated_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "problem_type": "single_label_classification",
  "structure": [],
  "torch_dtype": "float32",
  "torchscript": true,
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file tinybert/sentiment/updated_model/pytorch_model.bin
All model checkpoint weights 

In [6]:
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)

  0%|          | 0/8346 [00:00<?, ?ex/s]

  0%|          | 0/2087 [00:00<?, ?ex/s]

In [32]:
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}', evaluate = True)

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 8346
  Batch size = 32


{'eval_loss': 0.2261132448911667,
 'eval_accuracy': 0.917685118619698,
 'eval_runtime': 3.2137,
 'eval_samples_per_second': 2597.032,
 'eval_steps_per_second': 81.216}

In [10]:
encoded = tokenizer.encode("i love movies", add_special_tokens=True, return_tensors="pt").to(device)
traced_model = torch.jit.trace(model, encoded)

In [11]:
torch.jit.save(traced_model, f"{folder_name}/{dataset_name}/traced_m.pt")

In [12]:
traced_model = torch.jit.load(f"{folder_name}/{dataset_name}/traced_m.pt")

In [13]:
predict_sentences([tokenizer.tokenize("i hate movies")])

tensor([[ 1.7911, -1.7094]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([0])

In [14]:
predict_sentences([tokenizer.tokenize("i love movies")])

tensor([[-2.3436,  2.2181]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([1])